## Markov chain Monte Carlo

Based on a project from Physics 151.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

***

#### MCMC chain

When working in 'high' dimensional spaces it's often difficult to visualize likelihood surfaces, look at projections of the likelihood or even store the likelihood in a useful way.  This is made more complex if you want to compute the expectation value or distribution of some complex function of your parameters.

A highly convenient way of handling all of these problems is a Markov Chain.  Such a chain consists of a sequence of values of your parameters (usually a file with one line of numbers for each $\vec{\theta}$) where the frequency in the chain is proportional to the probability of that model given the data.  With such a chain in hand many statistical problems become very straightforward.

Markov Chains are generated by Markov Chain Monte Carlo algorithms: general methods based on drawing values of $\theta$ from approximate distributions and then correcting those draws to better aproximate the target posterior distribution. The sampling is done sequentially, with the distribution of the sampled draws depending on the last value drawn - hence, the draws from a Markov chain. (p. 275, <i>Bayesian Data Analysis</i>, Andrew Gelman et al.; Remember that a sequence $x_1, x_2, ...$ of random events is called a Markov chain if $x_{n+1}$ depends explicitly only on $x_{n}$ and not on previous steps.)

More advanced algorithms can operate to some extent in parallel (see below).

***

###  Supernova Cosmology Project

We will use a compilation of supernovae data to show that the expansion of the universe is accelerating, and hence it contains dark energy. This was [Nobel prize winning research](https://www.nobelprize.org/nobel_prizes/physics/laureates/2011/), and Saul Perlmutter, a professor of physics at Berkeley, shared a prize in 2011 for this discovery.  The method was deceptively simple:

"The expansion history of the universe can be determined quite easily, using as a “standard candle” any distinguishable class of astronomical objects of known intrinsic brightness that can be identified over a wide distance range. As the light from such beacons travels to Earth through an expanding universe, the cosmic expansion stretches not only the distances between galaxy clusters, but also the very wavelengths of the photons en route. By the time the light reaches us, the spectral wavelength $\lambda$ has thus been redshifted by precisely the same incremental factor $z = \Delta \lambda/\lambda$ by which the cosmos has been stretched in the time interval since the light left its source. The recorded redshift and brightness of each such object thus provide a measurement of the total integrated expansion of the universe since the time the light was emitted. A collection of such measurements, over a sufficient range of distances, would yield an entire historical record of the universe’s expansion." (Saul Perlmutter, http://supernova.lbl.gov/PhysicsTodayArticle.pdf).

Type Ia Supernovae (which are thought to arise from the collapse of a white dwarf upon reaching the Chandrasekhar limit) are promising standard candles.

We can infer the "luminosity distance" $D_L=(1+z)\chi(z)$ from measuring the inferred brightness of a supernova of luminosity $L$.  In a flat Universe with matter density $\Omega_m$
$$
\chi(z) = c\int_0^z \frac{dz'}{H(z')}
\quad\mathrm{with}\quad
H^2(z) = H_0^2\left[\Omega_m(1+z)^3 + (1-\Omega_m)\right]
$$
Writing $H_0 = 100\, h\ [\mathrm{km}\, s^{-1}\mathrm{Mpc}^{-1}]$ the luminosity distance is
$$
  D_L = 2997.92458\mathrm{Mpc}\ \,h^{-1}(1+z)\int_0^z \frac{dz'}{[\Omega_m(1+z')^3 + (1-\Omega_m)]^{1/2}}
$$
If we express fluxes in magnitudes $m$, where $m = -2.5\mathrm{log}_{10}F$ + const, then distances can be written in terms of the distance modulus, $\mu = m - M$ ($M$ is the absolute magnitude, the value of $m$ if the supernova is at a distance 10pc). Then:
$$
  \mu = 25 + 5\log_{10}\left(\frac{D_L}{\mathrm{Mpc}}\right)
$$

We will use the [SCP Union2.1 Supernova (SN) Ia compilation](http://supernova.lbl.gov/union/) to measure $\Omega_m$.

In [ ]:
# First, load the measured data: z, $\mu$ , $\sigma(\mu)$
data = np.loadtxt("sn_z_mu_dmu_plow_union2.1.txt", usecols=range(1,5))
# z
z_data = data[:,0]
# mu
mu_data = data[:,1]
# error on mu (sigma(mu))
mu_err_data = data[:,2]

In [ ]:
# Plot the data and some models just as a sanity check.
# We'll assume h=0.7 for now.
def integrand_DL(z,Omegam):
    intg = 1.0/np.sqrt(Omegam*(1+z)**3 + (1-Omegam))
    return(intg)

def DL_integrate(z,Omegam):
    zval  = np.linspace(0.,z,1000)
    DLval = 2997.92458/0.7*(1.0+z)*np.trapz(integrand_DL(zval,Omegam),x=zval)
    return(DLval)

def mu_model(z,Omegam):
    mu_model = 25.0 + 5.0*np.log10(DL_integrate(z,Omegam))
    return(mu_model)

In [ ]:
z   = np.linspace(0.01, 1.5, 100)
Mu1 = np.zeros(len(z))
Mu2 = np.zeros(len(z))
Mu3 = np.zeros(len(z))

for i in range(len(z)):
    Mu1[i] = mu_model(z[i],0)
    
for i in range(len(z)):
    Mu2[i] = mu_model(z[i],0.3)

for i in range(len(z)):
    Mu3[i] = mu_model(z[i],1)
    
    
plt.figure(figsize = (10,6))
plt.errorbar(z_data, mu_data, yerr = mu_err_data, marker = 'o', mfc='crimson', mec='crimson', ecolor = 'crimson', elinewidth = 1.5, barsabove = True, capsize = 2.0,  linestyle = 'None', label = 'Data')
plt.semilogx(z, Mu1, linewidth = 2.5, label = '$\Omega_M = 0$')
plt.semilogx(z, Mu2, linewidth = 2.5, label = '$\Omega_M = 0.3$')
plt.semilogx(z, Mu3, linewidth = 2.5, label = '$\Omega_M = 1$')
plt.legend()
plt.xlim(0.01, 1.5)
plt.xlabel('$z$',fontsize=18)
plt.ylabel('$\mu$',fontsize=18)
plt.show()

We've anticipated their great discovery already by plotting flat models with a cosmological constant above:

"If these data are correct, the obvious implication is that the simplest cosmological model must be too simple. The next simplest model might be one that Einstein entertained for a time. Believing the universe to be static, he tentatively introduced into the equations of general relativity an expansionary term he called the “cosmological constant” ($\Lambda$) that would compete against gravitational collapse. After Hubble’s discovery of the cosmic expansion, Einstein famously rejected $\Lambda$ as his “greatest blunder.” In later years, $\Lambda$ came to be identified with the zero-point vacuum energy of all quantum fields. It turns out that invoking a cosmological constant allows us to fit the supernova data quite well." (Saul Perlmutter, https://www.nobelprize.org/nobel_prizes/physics/laureates/2011/)

Rather than assuming DE is a cosmological constant let's allow it to have a general "equation of state", $w=p_{DE}/\rho_{DE}$.  For simplicity we'll keep $w$ $z$-independent for now.  Then
$$
  H^2(z) = H_0^2[\Omega_m(1+z)^3 + \Omega_{DE}(1+z)^{3(1+w)} + (1-\Omega_m-\Omega_{DE})(1+z)^2]
$$
with the last term being curvature.  Taking $w = -1$ and $\Omega_m+\Omega_{DE}=1$ returns our earlier result.

(For simplicity we'll assume the absolute magnitude of the SNe and $H_0$ are known, though one can sample from those as well if desired.  Keeping them fixed gives us a nice 2D problem for visualization though.)

In [ ]:
# Generalize our above:
def integrand_DL(z,Omegam,OmegaDE,w):
    return (Omegam*(1+z)**3 + OmegaDE*(1+z)**(3*(1+w)) + (1-Omegam-OmegaDE)*(1+z)**2)**(-0.5)

def DL_integrate(z,Omegam,OmegaDE,w):
    zval  = np.linspace(0,z,1000)
    DLval = 2997.92458/0.7*(1.0+z)*np.trapz(integrand_DL(zval,Omegam,OmegaDE,w),x=zval)
    return DLval

def mu_model(z,Omegam,OmegaDE,w):
    mu_model = 25.0 + 5.0*np.log10(DL_integrate(z,Omegam,OmegaDE,w))
    return mu_model

In [ ]:
z = np.linspace(0.01, 1.5, 100)
Mu1 = np.zeros(len(z))
Mu2 = np.zeros(len(z))
Mu3 = np.zeros(len(z))

for i in range(len(z)):
    Mu1[i] = mu_model(z[i],0.3, 0, 0)
    
for i in range(len(z)):
    Mu2[i] = mu_model(z[i],0, 1., -1)

for i in range(len(z)):
    Mu3[i] = mu_model(z[i],0.3, 0.7, -1)
    
plt.figure(figsize = (12,6))
plt.errorbar(z_data, mu_data, yerr = mu_err_data, marker = 'o', mfc='crimson', mec='crimson', ecolor = 'crimson', elinewidth = 1.5, barsabove = True, capsize = 2.0,  linestyle = 'None', label = 'Data')
plt.semilogx(z, Mu1, linewidth = 2.5, label = '$\Omega_M = 0.3, \Omega_{\Lambda} = 0$')
plt.semilogx(z, Mu2, linewidth = 2.5, label = '$\Omega_M = 0, \Omega_{\Lambda} = 1$')
plt.semilogx(z, Mu3, linewidth = 2.5, label = '$\Omega_M = 0.3, \Omega_{\Lambda} = 0.7$')
plt.legend()
plt.xlim(0.01, 1.5)
plt.xlabel('$z$',fontsize=18)
plt.ylabel('$\mu$',fontsize=18)
plt.show()

This is essentially the "money plot" from the original SN paper (though with slightly updated data).

The $\Omega_m = 0.3$ and $\Omega_m = 0.7$ model fits the data best. In combination with the CMB data, this shows that about 70% of the total energy density is vacuum energy and 30% is mass.

Let's continue to a Bayesian MCMC analysis to estimate the cosmological parameters.

Let us assume that the universe is flat (which is a fair assumption given modern data).
Assuming that errors are Gaussian (can be justified by averaging over large numbers of SN; central limit theorem), we calculate the likelihood $L$ as:
$$
  L \propto \mathrm{exp}\Big( -\frac{1}{2} \sum_{i = 1}^{N_{\mathrm{SN}}} \frac{[\mu_{i,\ data}(z_i) - \mu_{i,\ model}(z_i, \Omega_m, w)]^2}{\sigma(\mu_i)^2} \Big)
$$
where $z_i$, $\mu_i$, $\sigma(\mu_i)$ are from the measurements, and we compute
$\mu_{model}$ as a function of $z$, $\Omega_m$ and $w$.

We will find the probability distribution of the data using the
[Metropolis algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

1. Set the random initial point in the parameter space $(w, \Omega_m)$: let $w$ be negative and $\Omega_m$ be positive and draw a random number using np.random.uniform(). Set initial likelihood to low value (e.g. -1.e100) so that next point is accepted.

2. Now, draw a new sample starting from this random initial point. Here we assume that the proposal distribution is Gaussian with arbitrary width: in this problem, we assume that $\sigma = 0.01$ (This determines how far you propose jumps.) for distributions for both $w$ and $\Omega_m$.

3. Now, evaluate the log likelihood value of this new point.

4. If the value has gone up, accept the point.

5. Otherwise, accept it with probability given by ratio of likelihoods: Draw a random number from a uniform distribution between 0 and 1 ( $\alpha$ = np.random.uniform() ). If the ratio $ln(\frac{L_{new}}{L_{old}})$ is greater than $ln(\alpha)$ (i.e. $\frac{L_{new}}{L_{old}} > \alpha$), then accept it. Otherwise, reject it and stay at your old point.

6. Repeat this 15,000 times (the length of chain) and plot the distributions of $(w, \Omega_m)$.

I've arbitrarily set the length of MCMC chain to be 15,000. In the end, you should throw away the first 20% of the chain as burn-in. (20% is an arbitrary number). You can plot the chain and estimate the burn-in period and make the chain longer if necessary.

__Note__ The implementation below is slow and painful, but is meant entirely for pedagogical reasons.  It takes a little while to run...

In [ ]:
# Import data
data = np.loadtxt("sn_z_mu_dmu_plow_union2.1.txt", usecols=range(1,5))
z_data,mu_data,mu_err_data = data[:,0],data[:,1],data[:,2]

# length of MCMC chain and number of parameters: Omega_m and w
nsamples,npars = 15000,2

# Define (gaussian) width of the proposal distribution, one for each parameter.
# This determines how far you propose jumps
Sigma = [0.01, 0.01]

# Declare an empty array of the parameter values of each point. 
# Theta[:,0] stores a trace of the parameter \Omega_m  
# Theta[:,1] stores a trace of the parameter w 
# Theta[:,2] stores log-likelihood values at each point
Theta = np.empty([nsamples,npars+1])

# Random starting point in parameter space
# Set initial likelihood to low value so next point is accepted (could compute it instead):
Theta[0,:] = [np.random.uniform(), -np.random.uniform(), -1.e100]
print("Initial params: ",Theta[0,:])

In [ ]:
# Define mu from theory
# Really want something fast here, which can operate on vectors
# of z's, so I'll use the fact that our integral turns out to
# be expressable as Hypergeometric functions.  We could also
# do power series expansions or Pade approximants or call some
# other library.
from scipy.special import hyp2f1
def DL_z(z,OmegaM,w):
    """Returns D_L(z) for flat const-w models using Gaussian hypergeometric functions."""
    zp13w = (1+z)**(3*w)
    fac   = (OmegaM-1)/OmegaM
    a,b,c = 0.5,-1./6./w,1-1./6./w
    t1    = hyp2f1(a,b,c,fac)
    t2    = hyp2f1(a,b,c,fac*zp13w)
    tmp   = (t1-t2/np.sqrt(1+z))
    tmp   = 2997.925/0.7*(1+z) * 2/np.sqrt(OmegaM)*tmp
    return(tmp)
#
def mu_model(z,Omegam,w):
    mu_model = 25.0 + 5.0*np.log10(DL_z(z,Omegam,w))
    return(mu_model)

In [ ]:
# Define the likelihood function:
import math
def lnL(Omegam,w):
    # Treat unphysical regions by setting likelihood to (almost) zero:    
    if(Omegam<=0 or w>=0):
        lnL = -1.e100
    else:
        # Compute difference with theory mu at redshifts of the SN, for trial Omegam
        Dmu = mu_data - mu_model(z_data,Omegam,w)
        # Compute ln(likelihood) assuming gaussian errors
        lnL = -0.5*np.sum((Dmu/mu_err_data)**2)
    return lnL


# Draw new proposed samples from a proposal distribution, centred on old values Omegam[i-1]
# Accept or reject, and colour points according to ln(likelihood):

# Compute initial likelihood value:
Theta[0,npars] = lnL(Theta[0,0], Theta[0,1])
print("Initial Likelihood",Theta[0,npars])

progress = nsamples/10; val = 0
for i in range(1,nsamples):    
    
    if i%progress == 0:
        val = val + 10
        print("%d percent done" %val)
    
    lnLPrevious = Theta[i-1,npars]
    OmegamProp = np.random.normal(Theta[i-1,0],Sigma[0])
    wProp = np.random.normal(Theta[i-1,1],Sigma[1])
    
    lnLProp    = lnL(OmegamProp, wProp)

    # Metroplis-Hastings algorithm:

    if(lnLProp > lnLPrevious):
    # Accept point if likelihood has gone up:
        Theta[i,0]     = OmegamProp
        Theta[i,1]     = wProp
        Theta[i,npars] = lnLProp
    else:
    # Otherwise accept it with probability given by ratio of likelihoods:
        alpha = np.random.uniform()
    
        if(lnLProp - lnLPrevious > np.log(alpha)):
            Theta[i,0]     = OmegamProp
            Theta[i,1]     = wProp
            Theta[i,npars] = lnLProp
        else:
        # Reject; Repeat the previous point in the chain:
            Theta[i,0:2]     = Theta[i-1,0:2]
            Theta[i,npars] = lnLPrevious

# Remove a burn in period, arbitrarily chosen to be the first 20% of the chain:
nburn = 2*math.floor(nsamples/10)
    

# Plot the histogram of Omegam after the burn-in phase:
plt.hist(Theta[nburn:,0],bins=30)
plt.xlabel(r'$\Omega_m$',fontsize=18)
plt.ylabel(r'Probability',fontsize=18)
plt.show()

# Plot the histogram of w after the burn-in phase:
plt.hist(Theta[nburn:,1],bins=30)
plt.xlabel('w',fontsize=18)
plt.ylabel(r'Probability',fontsize=18)
plt.show()

# Scatter plot of the samples (2-d posterior):
plt.scatter(Theta[nburn:,0], Theta[nburn:,1], c = -Theta[nburn:,npars])
plt.xlabel(r'$\Omega_m$',fontsize=18)
plt.ylabel('w',fontsize=18)
plt.show() 

# Print best-fit values and constraints
print ('Omega_m = ',np.mean(Theta[nburn:,0]), '+/-' ,np.std(Theta[nburn:,0]))
print ('w = ',np.mean(Theta[nburn:,1]), '+/-' ,np.std(Theta[nburn:,1]))

***

## Convergence ##

So how do we know we've run enough samples?

We need to make sure that chains converge to the posterior distribution. One useful test for convergence is "Gelman-Rubin statistic." For a given parameter, $\theta$, the $R$ statistic compares the variance across chains with the variance within a chain. Intuitively, if the chains are random-walking in very different places, i.e. not sampling the same distribution, $R$ will be large.

In detail, given chains $J=1,\ldots,m$, each of length $n$:
* Let $B=\frac{n}{m-1} \sum_j \left(\bar{\theta}_j - \bar{\theta}\right)^2$, where $\bar{\theta_j}$ is the average $\theta$ for chain $j$ and $\bar{\theta}$ is the global average. This is proportional to the variance of the individual-chain averages for $\theta$.
* Let $W=\frac{1}{m}\sum_j s_j^2$, where $s_j^2$ is the estimated variance of $\theta$ within chain $j$. This is the average of the individual-chain variances for $\theta$.
* Let $V=\frac{n-1}{n}W + \frac{1}{n}B$. This is an estimate for the overall variance of $\theta$.
* Finally, $R=\sqrt{\frac{V}{W}}$.

We'd like to see $R\approx 1$ (e.g. $R < 1.1$ is often used). Note that this calculation can also be used to track convergence of combinations of parameters, or anything else derived from them. 

Reference: https://github.com/KIPAC/StatisticalMethods/blob/master/chunks/montecarlo1.ipynb

Let us compute $R$ and determine if the condition $R < 1.1$ is satisfied.

In [ ]:
def create_MCMC_chain():
    # Make a new chain.
    Theta = np.empty([nsamples,npars+1])
    # Random starting point in parameter space
    Theta[0,:] = [np.random.uniform(), -np.random.uniform(), -1.e100]
    # Compute initial likelihood value:
    Theta[0,npars] = lnL(Theta[0,0], Theta[0,1])
    for i in range(1,nsamples):    
        lnLPrevious = Theta[i-1,npars]
        OmegamProp = np.random.normal(Theta[i-1,0],Sigma[0])
        wProp = np.random.normal(Theta[i-1,1],Sigma[1])
        lnLProp    = lnL(OmegamProp, wProp)
        #
        # Metroplis-Hastings algorithm:
        if(lnLProp > lnLPrevious):
        # Accept point if likelihood has gone up:
            Theta[i,0]     = OmegamProp
            Theta[i,1]     = wProp
            Theta[i,npars] = lnLProp
        else:
        # Otherwise accept it with probability given by ratio of likelihoods:
            alpha = np.random.uniform()
    
            if(lnLProp - lnLPrevious > np.log(alpha)):
                Theta[i,0]     = OmegamProp
                Theta[i,1]     = wProp
                Theta[i,npars] = lnLProp
            else:
            # Reject; Repeat the previous point in the chain:
                Theta[i,0:2]     = Theta[i-1,0:2]
                Theta[i,npars] = lnLPrevious
    # Remove a burn in period, arbitrarily chosen to be the first 20% of the chain:
    nburn = 2*math.floor(nsamples/10)
    return(Theta[nburn:,:])

In [ ]:
chain1 = create_MCMC_chain()
chain2 = create_MCMC_chain()

In [ ]:
# Let's do OmegaM.
n,m = chain1.shape[0],2
OmM1= np.mean(chain1[:,0])
OmM2= np.mean(chain2[:,0])
OmM = np.mean([chain1[:,0],chain2[:,0]])
B   = n/(m-1.0)*( (OmM1-OmM)**2+(OmM2-OmM)**2 )
W   = np.mean([np.var(chain1[:,0]),np.var(chain2[:,0])])
V   = (n-1.)/n*W + B/n
R   = np.sqrt(V/W)
print(R)

Which is safely less than 1.1, so we ran enough samples (we would probably combine those two chains into one).  Depending on how big the chains were, we could also "thin" them by taking every few samples rather than keeping all of the samples.  If you look at the chain you'll find it tends to have long streches of the same value (where nothing was accepted).  One way to see how "long" your chain is and how "thinned" it could be is to compare lengths to the "correlation length".

The autocorrelation of a sequence, as a function of lag, $k$, is defined thusly:
$$
\rho_k = \frac{\sum_{i=1}^{n-k}\left(\theta_{i} - \bar{\theta}\right)\left(\theta_{i+k} - \bar{\theta}\right)}{\sum_{i=1}^{n-k}\left(\theta_{i} - \bar{\theta}\right)^2} = \frac{\mathrm{Cov}_i\left(\theta_i,\theta_{i+k}\right)}{\mathrm{Var}(\theta)}
$$
The larger lag one needs to get a small autocorrelation, the less informative individual samples are.  The correlation length can be defined as how far apart samples have to be for the correlation to drop to a certain fraction of the zero-lag value.

An alternative to the autocorrelation of the sequence is to plot its power spectrum (i.e. Fourier transform squared).  If the samples were uncorrelated the power spectrum would be flat (i.e. white noise).  So we expect to see a long, flat piece of P(k) and then less power at high k because the samples are highly correlated.  The length of the flat segment is basically how many "independent" samples you have in the chain.

In [ ]:
chain1_ft = np.fft.fft(chain1[:,0]-np.mean(chain1[:,0]))[:100]
chain1_Pk = np.abs(chain1_ft)**2
plt.semilogx(chain1_Pk)
plt.xlabel(r'$k/k_{\rm fund}$',fontsize=18)
plt.ylabel(r'$P(k)$',fontsize=18)

So we have less than O(10) "fully sampled" pieces in this chain.  Long enough to be giving us a fair sample, but not overkill.

## Canned packages ##

It's not much work to write your own MCMC code, even using the more modern algorithms.  However, the cosmology community primarily uses two packages: [CosmoMC](https://cosmologist.info/cosmomc/), which is usually integrated with CAMB, and [emcee](http://dfm.io/emcee/current/) which uses the more modern Goodman & Weare affine invariant ensemble sampler.  Personally I like the Goodman & Weare sampler because it (a) doesn't require lots of tuning of the proposal distribution and (b) allows modest amounts of parallelization (e.g. up to hundreds of "walkers").  It is a more modern take on MCMC theory than what's coded in CosmoMC.